In [1]:
import subprocess
import time
import re

## Function for command compatibility with windows cmd and some linux commands

In [2]:
def run_cmd(cmd, **kwargs):
    response = subprocess.Popen(
                    cmd, 
                    stderr=subprocess.PIPE, 
                    stdout=subprocess.PIPE, 
                    **kwargs)

    stdout, stderr = response.communicate()
    stdout = stdout.decode()
    stderr = stderr.decode()

    returncode = response.returncode

    # print('STDOUT BEGIN \n\n', stdout, '\nSTDOUT END\n\n\n')

    # If had parameter shell=True ignore bug of stderr on windows
    if kwargs.get('shell', False):
        return stdout
        
    else:
        if returncode != 0:
            print(f'Error code:   {returncode}')
            print(stderr)
        else:
            return stdout

## Verifying that devices are available

In [3]:
def _devices_connected():
    cmd = 'adb devices -l | find "device " /c'
    if int(run_cmd(cmd, shell=True)) > 0:
        return True
    else:
        return False

## Obtaining the device data from the device list of the adb command

In [4]:
def get_devices_data():
    if not _devices_connected():
        return None

    cmd = 'adb devices -l'
    devices_list = run_cmd(cmd).strip().split('\n')[1:]

    for index, device in enumerate(devices_list):

        device = device.strip().split(' ')
        device.reverse()

        if device[4] != 'device':
            continue
        
        device_data = { data.split(':')[0] : data.split(':')[1] for data in device[0:4] }

        device_data['available'] = True if device[4] == 'device' else False
        device_data['serial'] = device[-1]

        devices_list[index] = device_data
    
    return devices_list

## Shortcut to run command in ADB

In [5]:
def run_adb(cmd, serial=None):
    if not serial:
        devices = get_devices_data()
        if not devices:
            return None
        else:
            serial = devices[0]['serial']

    sh = f'adb -s {serial} shell '
    return run_cmd(sh + cmd)


## Connecting to the device console to get its IP address

In [6]:
def get_device_ip(serial = None):
    if serial is None:
        serial = get_devices_data()[0]['serial']

    cmd = '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

    return run_adb(cmd).split('/')[0]

In [7]:
def tcpip_connect(port = 5555, serial = None, ip = None):
    port = str(port)
    ip = get_device_ip(serial) if ip is None else ip
    
    cmd = f'adb tcpip {port}' if serial is None else f'adb -s {serial} tcpip {port}'
    print(run_cmd(cmd))

    cmd = f'adb connect {ip}:{port}'
    print(run_cmd(cmd))


In [8]:
tcpip_connect()
time.sleep(2)


restarting in TCP mode port: 5555

already connected to 192.168.1.67:5555



In [9]:
devices_list = get_devices_data()
for index, device in enumerate(devices_list):
    devices_list[index]['ip'] = get_device_ip(device['serial'])

devices_list

[{'transport_id': '9',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555',
  'ip': '192.168.1.67'}]

## Perform some stuff of manteniance

In [10]:
serial = devices_list[0]['serial']
path = r'/sdcard/musica/AC\ DC/'

serial, path

('192.168.1.67:5555', '/sdcard/musica/AC\\ DC/')

In [11]:
# Searching for all the files that do not have the extension .mp3, .m4a and .flac (without distinguishing capitals)
cmd = f'find {path} -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac | wc -l'
print(run_adb(cmd, serial))

# Deleting that files
cmd = f'find {path} -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac -delete'
print(run_adb(cmd, serial))

cmd = f'find {path} -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac | wc -l'
print(run_adb(cmd, serial))


1

/sdcard/musica/AC DC/test/test.sh

0



In [12]:
# Now with empty folders
cmd = f'find {path} -type d -empty | wc -l'
print(run_adb(cmd, serial))

cmd = f'find {path} -type d -empty'
print(run_adb(cmd, serial))

cmd = f'find {path} -type d -empty -delete'
print(run_adb(cmd, serial))

cmd = f'find {path} -type d -empty | wc -l'
print(run_adb(cmd, serial))

0



0



In [13]:
cmd = f'ls -p {path} -R'
artist_content = run_adb(cmd, serial).strip()
artist_content

"/sdcard/musica/AC DC/:\r\n74 Jailbreak (1984)/\r\nAC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]/\r\nBack In Black (Australian Editions) (1980)/\r\nBacktracks (2009)/\r\nBallbreaker (1995)/\r\nBlack Ice (2008)/\r\nBlow Up Your Video (Australian Editions) (1988)/\r\nDirty Deeds Done Dirt Cheap (Australian Edition) (1976)/\r\nFlick Of The Switch (Australian Editions) (1983)/\r\nFly On The Wall (Australian Editions) (1985)/\r\nFor Those About To Rock (We Salute You) (1981)/\r\nHigh Voltage (Australian Editions) (1975)/\r\nHighway To Hell (Australian Editions) (1979)/\r\nIf You Want Blood You've Got It (Australian Editions) (1978)/\r\nPowerage (Australian Editions) (1978)/\r\nStiff Upper Lip (2000)/\r\nT.N.T (Australian Editions) (1975)/\r\nThe Razor's Edge (Australian Editions) (1990)/\r\nWho Made Who (Australian Editions) (1986)/\r\ntest/\r\n\r\n/sdcard/musica/AC DC/74 Jailbreak (1984):\r\n01 - Jailbreak.mp3\r\n02 - You Aint Got A Hold On Me.mp3\r\n03 - Show Business.mp3\r\n04 - Soul Stri

In [14]:
artist_content = artist_content.replace('\n', '')
# Adding '\' to special characters
artist_content = re.sub(r'(\()|(\))|(\')|( )', r'\\\g<0>', artist_content)
artist_content = artist_content.split('\r')

# deleting empty items
artist_content = [item for item in artist_content if item]

# Extracting only folders
folders = []
for index, item in enumerate(artist_content):
    if item[-1] == ':':
        folders.append( (item[:-1] + '/', index) )

# Insert files into each folder
tree = {}
for index, folder in enumerate( folders ):
    try:
        next_folder = folders[index + 1]
        slice_end = next_folder[1]
    except IndexError:
        slice_end = len(artist_content)
    slice_begin = folder[1] + 1
    if slice_end - slice_begin == 1:
        continue    
    else:
        key = folder[0]
    value = artist_content[slice_begin:slice_end]
    tree[ key ] = value

# Droping first key, that include primary folder with only subfolders
tree.pop(path + '/', None)

tree

{'/sdcard/musica/AC\\ DC/74\\ Jailbreak\\ \\(1984\\)/': ['01\\ -\\ Jailbreak.mp3',
  '02\\ -\\ You\\ Aint\\ Got\\ A\\ Hold\\ On\\ Me.mp3',
  '03\\ -\\ Show\\ Business.mp3',
  '04\\ -\\ Soul\\ Stripper.mp3',
  '05\\ -\\ Baby\\ Please\\ Dont\\ Go.mp3'],
 '/sdcard/musica/AC\\ DC/AC_DC\\ -\\ Rock\\ or\\ Bust\\ \\(2014\\)\\ [MP3\\ @\\ 320\\ kbps]/': ['01.\\ Rock\\ or\\ Bust.mp3',
  '04.\\ Miss\\ Adventure.mp3',
  '05.\\ Dogs\\ of\\ War.mp3',
  '08.\\ Baptism\\ By\\ Fire.mp3',
  '09.\\ Rock\\ The\\ House.mp3',
  '10.\\ Sweet\\ Candy.mp3'],
 '/sdcard/musica/AC\\ DC/Back\\ In\\ Black\\ \\(Australian\\ Editions\\)\\ \\(1980\\)/': ['01\\ Hells\\ Bells.mp3',
  '02\\ Shoot\\ To\\ Thrill.mp3',
  '04\\ Giving\\ The\\ Dog\\ A\\ Bone.mp3',
  '05\\ Let\\ Me\\ Put\\ My\\ Love\\ Into\\ You.mp3',
  '06\\ Back\\ In\\ Black.mp3',
  '07\\ You\\ Shook\\ Me\\ All\\ Night\\ Long.mp3',
  '08\\ Have\\ A\\ Drink\\ On\\ Me.mp3',
  '09\\ Shake\\ A\\ Leg.mp3',
  "10\\ Rock\\'n\\'\\ Roll\\ Ain\\'t\\ Noise\\ Pollution.

In [15]:
# Rename files ( Deleting index number )
tree_modify = {}
search = re.compile(r'^\d+[\\ . \s -]+')
rename_count = 0
same_count = 0

for folder in tree:
    files = []
    for file in tree[folder]: 
        modify = search.subn('', file)
        files.append(modify[0])
        if modify[1] > 0:
            rename_count += 1
        else:
            same_count += 1
    tree_modify[folder] = files
    
print(f'{rename_count} files renamed!!')
print(f'{same_count} files still same name\n')
for folder, files in tree_modify.items():
    print(folder)
    for file in files:
        print(f'\t{file}') 
        
    

104 files renamed!!
14 files still same name

/sdcard/musica/AC\ DC/74\ Jailbreak\ \(1984\)/
	Jailbreak.mp3
	You\ Aint\ Got\ A\ Hold\ On\ Me.mp3
	Show\ Business.mp3
	Soul\ Stripper.mp3
	Baby\ Please\ Dont\ Go.mp3
/sdcard/musica/AC\ DC/AC_DC\ -\ Rock\ or\ Bust\ \(2014\)\ [MP3\ @\ 320\ kbps]/
	Rock\ or\ Bust.mp3
	Miss\ Adventure.mp3
	Dogs\ of\ War.mp3
	Baptism\ By\ Fire.mp3
	Rock\ The\ House.mp3
	Sweet\ Candy.mp3
/sdcard/musica/AC\ DC/Back\ In\ Black\ \(Australian\ Editions\)\ \(1980\)/
	Hells\ Bells.mp3
	Shoot\ To\ Thrill.mp3
	Giving\ The\ Dog\ A\ Bone.mp3
	Let\ Me\ Put\ My\ Love\ Into\ You.mp3
	Back\ In\ Black.mp3
	You\ Shook\ Me\ All\ Night\ Long.mp3
	Have\ A\ Drink\ On\ Me.mp3
	Shake\ A\ Leg.mp3
	Rock\'n\'\ Roll\ Ain\'t\ Noise\ Pollution.mp3
/sdcard/musica/AC\ DC/Backtracks\ \(2009\)/CD1\ Studio\ Rarities/
	A01\ Stick\ Around.mp3
	A03\ Fling\ Thing.mp3
	A04\ R.I.P.\ \(Rock\ In\ Peace\).mp3
	A06\ Crabsody\ In\ Blue.mp3
	A09\ Borrowed\ Time.mp3
	A10\ Down\ On\ The\ Borderline.mp3
	A11\

In [16]:
# Verify same len of both trees
for i in tree:
    len_tree = len(tree[i])
    len_tree_1 = len(tree_modify[i])
    equal = True if len_tree == len_tree_1 else False
    print(i)
    print(f'Tree: {len_tree}   Tree_1: {len_tree_1}    Equals:  {equal}\n')

/sdcard/musica/AC\ DC/74\ Jailbreak\ \(1984\)/
Tree: 5   Tree_1: 5    Equals:  True

/sdcard/musica/AC\ DC/AC_DC\ -\ Rock\ or\ Bust\ \(2014\)\ [MP3\ @\ 320\ kbps]/
Tree: 6   Tree_1: 6    Equals:  True

/sdcard/musica/AC\ DC/Back\ In\ Black\ \(Australian\ Editions\)\ \(1980\)/
Tree: 9   Tree_1: 9    Equals:  True

/sdcard/musica/AC\ DC/Backtracks\ \(2009\)/CD1\ Studio\ Rarities/
Tree: 8   Tree_1: 8    Equals:  True

/sdcard/musica/AC\ DC/Ballbreaker\ \(1995\)/
Tree: 7   Tree_1: 7    Equals:  True

/sdcard/musica/AC\ DC/Black\ Ice\ \(2008\)/
Tree: 8   Tree_1: 8    Equals:  True

/sdcard/musica/AC\ DC/Blow\ Up\ Your\ Video\ \(Australian\ Editions\)\ \(1988\)/
Tree: 5   Tree_1: 5    Equals:  True

/sdcard/musica/AC\ DC/Dirty\ Deeds\ Done\ Dirt\ Cheap\ \(Australian\ Edition\)\ \(1976\)/
Tree: 3   Tree_1: 3    Equals:  True

/sdcard/musica/AC\ DC/Flick\ Of\ The\ Switch\ \(Australian\ Editions\)\ \(1983\)/
Tree: 4   Tree_1: 4    Equals:  True

/sdcard/musica/AC\ DC/Fly\ On\ The\ Wall\ \(Austr

## Changing the names of the tree by those of the tree_modify (Making the changes on the device)

In [18]:
for folder in tree:
    for index, file in enumerate(tree[ folder ]):
        new_file = tree_modify[ folder ][ index ]
        path = folder + '{} '
        cmd = 'mv ' + path.format(file) + path.format(new_file)
        run_adb(cmd)